# Dùng get-news để crawl dữ liệu tin tức

## 0. Change log
Ver0.1: Sử dụng thư viện [news-please](https://github.com/fhamborg/news-please), nhược điểm chưa lấy được URL của hình ảnh và videos

Ver0.2: Sử dụng [fake-useragent](https://pypi.org/project/fake-useragent/) để tạo _rotating proxy_ để có thể hỗ trợ lấy tin

Ver0.3: Sử dụng thêm thư viện BeautifulSoup để lấy thêm được URL của ảnh

## 1. Thư viện và các function

In [8]:
import json, os
from newsplease import NewsPlease
from bs4 import BeautifulSoup
from fake_useragent import UserAgent

from urllib.request import Request, urlopen

ua = UserAgent()

### 1.1. Các function liên quan đến tin tức
Từ các url tin tức, chúng ta đưa chúng vào trong 1 list, hàm này dùng thư viên _NewsPlease_ để đưa dữ liệu tin tức lấy được vào 1 list

In [9]:
def get_news_to_list(url_list):
    news_list = []
    for url in url_list:
        try:
            news_list.append(NewsPlease.from_url(url))
        except Exception as e: 
            print(e, ': ', url)
            continue
    return news_list

Đưa các dữ liệu sẵn có vào file JSON với cấu trúc là ```<json_prefix>_<json_no>.json```

Các thông số cho json.dump lần lượt là: __default = str__ đưa tất cả về string để tránh lỗi trong quá trình parse dữ liệu dạng _datetime_ sang json của các trường date\_modify, date\_download, date\_publish; __ensure\_ascii=False__ là để giữ các dữ liệu text ở dạng Unicode - để không bị lỗi khi đưa vào file json khi để text ở định dạng Unicode, thì trong phần __open()__ đặt thông số _encoding='utf-8'_. 

Các biến:
1. file\_name: tên file để lưu
2. file\_no: số thứ tự của file

In [10]:
def dump_news_to_json(news_list, json_begin_no, json_prefix, file_path):
    file_no = json_begin_no
    for news in news_list:
        file_name = json_prefix + str(file_no) + '.json' 
        file_no += 1
        with open(os.path.join(file_path, file_name), 'w', encoding='utf-8') as outfile:
            json.dump(news.__dict__, outfile, indent=4, sort_keys=True, default=str, ensure_ascii=False)

### 1.2. Các function liên quan đến việc tạo rotating proxy
Tham khảo tại một số trang sau:
1. [Create a rotating proxy crawler in Python 3](https://codelike.pro/create-a-crawler-with-rotating-ip-proxy-in-python/)

In [ ]:
def random_proxy(proxies):
  return random.randint(0, len(proxies) - 1)

In [ ]:
def retrive_proxies():
    proxies_req = Request('https://sslproxies.org/')
    proxies_req.add_header('User-Agent', ua.random)
    proxies_doc = urlopen(proxies_req).read().decode('utf8')
    
    

## 2. Các dữ liệu tin tức giả
URL về Fake News, tập hợp từ các bài viết có nội dung không được xác minh về nguồn tin, hoặc được xác nhận sai,... Có thể lưu url các bài báo trong 1 file text (mỗi url là 1 dòng)

In [16]:
fake_news_urls = [
    "https://www.vietgiaitri.com/phap-luat/201304/yeu-rau-xanh-p51-cau-chuyen-dong-troi-sau-tam-ri-do-mong-735408"
]

Bắt đầu crawl dữ liệu tin tức giả đưa vào list fake_articles

In [17]:
fake_articles = get_news_to_list(fake_news_urls)

<urlopen error EOF occurred in violation of protocol (_ssl.c:1045)> :  https://www.vietgiaitri.com/phap-luat/201304/yeu-rau-xanh-p51-cau-chuyen-dong-troi-sau-tam-ri-do-mong-735408


Tên của các file json sẽ chứa nội dung vừa crawl

In [14]:
fake_no = 100
fake_prefix = 'VFND_Ac_Fake_'

Đưa dữ liệu đã lấy vào các file JSON để lưu trữ

In [15]:
dump_news_to_json(fake_articles, fake_no, fake_prefix, './')

Test các kết quả trả ra khi crawl ở cell dưới

In [ ]:
print (fake_articles[0].__dict__)

## 3. Các dữ liệu tin tức thật

In [14]:
real_news_urls = [
                    
"https://thethao.tuoitre.vn/danh-bai-dokovic-zverev-lan-dau-vo-dich-giai-quan-vot-atp-finals-20181119055342346.htm",
"https://thethao.tuoitre.vn/hoa-hau-quan-vot-bouchard-tro-tai-da-bong-2018112221552504.htm",
"https://thethao.tuoitre.vn/quan-vot-vn-coi-chung-lac-duong-nhu-thai-lan-821535.htm",
"https://thethao.tuoitre.vn/federer-dung-dokovic-o-ban-ket-giai-quan-vot-paris-masters-20181103062334363.htm",
"https://thethao.tuoitre.vn/nadal-gap-tsitsipas-o-chung-ket-giai-quan-vot-barcelona-mo-rong-1439844.htm",
"https://dantri.com.vn/phap-luat/khoi-to-cap-nu-quai-chuyen-cung-cap-ma-tuy-thuoc-lac-20181209204749864.htm",
"https://dantri.com.vn/phap-luat/dung-dao-dam-chet-nguoi-vi-xich-mich-luc-do-xang-20181209202623487.htm",
"https://dantri.com.vn/phap-luat/quen-khoa-o-to-nu-tai-xe-bi-khoang-sach-tai-san-20181209143646284.htm",
"https://dantri.com.vn/phap-luat/lua-ban-be-gai-14-tuoi-bat-luon-nguoi-giai-cuu-de-doi-tien-chuoc-20181209122608572.htm",
"https://dantri.com.vn/phap-luat/doi-no-60-trieu-khong-duoc-ep-nan-nhan-viet-giay-vay-400-trieu-20181209083854909.htm",
"https://dantri.com.vn/phap-luat/vu-chay-than-chet-nguoi-cuu-giam-doc-benh-vien-bi-truy-to-tu-3-12-nam-tu-20181209083714518.htm",
"https://dantri.com.vn/phap-luat/sau-ly-hon-vo-tinh-nhan-doi-chia-tay-nen-ra-tay-sat-hai-20181209082922084.htm",
"https://dantri.com.vn/phap-luat/duoi-theo-ke-nem-chat-ban-chu-nha-bi-gay-chan-2018120908113454.htm",
"https://dantri.com.vn/phap-luat/doi-tuong-cuoi-cung-trong-vu-tron-khoi-trai-tam-giam-kien-giang-da-bi-bat-20181209081958807.htm",
"https://dantri.com.vn/the-thao/sieu-may-bay-a350-se-cho-cdv-viet-nam-di-malaysia-co-vu-chung-ket-luot-di-20181208082125017.htm",
"https://dantri.com.vn/the-thao/thuong-20000-usd-cho-doi-tuyen-co-vua-viet-nam-tai-olympiad-2018-20181017151513152.htm",
"https://dantri.com.vn/the-thao/truong-son-gianh-hcv-tai-giai-co-vua-dong-doi-the-gioi-2018-20181005221940293.htm",
"https://dantri.com.vn/the-thao/chuyen-ve-chang-sinh-vien-luat-kien-tuong-le-tuan-minh-20180320085855246.htm",
"https://dantri.com.vn/the-thao/tien-dao-malaysia-toi-da-co-cach-vuot-qua-hang-thu-doi-tuyen-viet-nam-20181209191129676.htm"
                 ]

Bắt đầu crawl dữ liệu, sử dụng thư viện news-please, các tin tức sẽ được lấy theo URL và được chứa trong mảng _real_\__article_.

In [15]:
real_articles = get_news_to_list(real_news_urls)

Do thư viện _news-please_ không hỗ trợ để lấy hình ảnh vì thế chúng ta sẽ lấy hình ảnh thông qua thư viện _BeautifulSoup_. Sau mỗi lần lấy chúng ta tạo ra 1 đối tượng dictionary cho mỗi URL

In [ ]:
real_imgs = []
for real_url in real_news_urls:
    

Merge 2 đối tượng dictionary. Tham khảo từ: [How to merge two dictionaries in a single expression? - StackOverflow](https://stackoverflow.com/questions/38987/how-to-merge-two-dictionaries-in-a-single-expression)

Tương tự với việc crawl dữ liệu về tin tức giả. Ta có danh sách 1 số biến:
1. real\_name: tên file để lưu
2. real\_no: số thứ tự của file

In [16]:
real_no = 105
real_prefix = 'VFND_Ac_Real_'

In [17]:
for real in real_articles:
    real_name = real_prefix + str(real_no) + '.json'
    real_no = real_no + 1
    with open(real_name, 'w', encoding='utf-8') as outfile:
        json.dump(real.__dict__, outfile, indent=4, sort_keys=True, default=str, ensure_ascii=False)

Test các kết quả trả ra khi crawl ở cell dưới

In [ ]:
print(real_articles[0].__dict__)